In [28]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Data

In [29]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TFIDF

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

C:\Users\acer\miniconda3\envs\jcop_ud\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# TFIDF Similarity -> Document Similarity

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
similarity

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [42]:
similarity.argsort() #dari yang tidak mirip sampai mirip

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

# Paling mirip

In [43]:
df.teks[0][:200] #dokumen ke 0 dengan 200 kata pertama

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [44]:
df.teks[144][:200] #dokumen ke 0 dengan 200 kata pertama

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

# Keyword Extraction

Ide:
TFIDF telah membobotkan kata2 yang spesifik kuat, sehingga tfidf sudah mengamnbil keywordnya. jadi tinggal sorting tfidf

In [45]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [52]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [53]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274201, 365453, 365452, ..., 386361, 436633, 169209], dtype=int64)

In [54]:
vocab[169209]

'ginandjar'

In [56]:
vocab[274201]

'litbang partai'

In [59]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [60]:
def extract_keywords_tfidf(doc, tfidf, topk = 10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()
    
    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [63]:
kompas1 = """"Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KGinandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan K"""

# kalo mau masukin text pakai kutip 3 ("""...""") agar tidak error ketika ada tanda kutip dalam text. kutip 3 itu ambil semua text walaupun di enter.contoh dibawah

In [65]:
"sdsdsd
dsdsd"

""dsdsd
dssdsd
s""

"""dsd
ds
d
sd
s
d"""

SyntaxError: EOL while scanning string literal (<ipython-input-65-e94e6561d5ad>, line 1)

In [64]:
extract_keywords_tfidf(kompas1, tfidf)

['agung dilaporkan',
 'dilaporkan polri',
 'kartasasmita batas',
 'ginandjar',
 'ditahan jaksa',
 'agung memutuskan',
 'korupsi ginandjar',
 'polri kejaksaan',
 'batas ditentukan',
 'memutuskan menahan']

# kalo mau liat text paling langka, pakai idf

In [67]:
tfidf.idf_.argsort()
#yang paling langka yang paling besar jadi yang paling kanan (ke kanan makin besar)

array([ 28062,      0, 200981, ..., 199875, 199906, 548403], dtype=int64)

In [68]:
vocab[548403]

'zx diserbu'

In [69]:
vocab[28062]

'``'